### Another example of using Router Chain or MultiPromptChain

In [5]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain_openai import ChatOpenAI 
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
postive_template =""" You are a helpful AI assistant focus on the positive side of things.
Whenever you analyze a text, you look for the positive aspects and hightlight them.
Here is the text
{input}"""

negative_template =""" You are a helpful AI assistant focus on the negative side of things.
Whenever you analyze a text, show potential downsides.
Here is the text
{input}"""

neutral_template =""" You are a helpful AI assistant focus on the neutral side of things.
Whenever you analyze a text, show potential neutral points, not favouring any positive or negative aspects.
Here is the text
{input}"""  

In [3]:
prompt_infos = [
    {
        "name":"positive",
        "description":"Good for analyzing positive sentiments",
        "prompt_template":"positive_template"
    },
    {
        "name":"negative",
        "description":"Good for analyzing negative sentiments",
        "prompt_template":"negative_template"
    },
    {
        "name":"neutral",
        "description":"Godd for analyzing neutral sentiments",
        "prompt_template":"neutral_template"
    }
]

In [6]:
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.4)

In [9]:
destination_chains ={}
for p_info in prompt_infos:
    name =p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template = prompt_template, input_variables=["input"])
    llm_chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = llm_chain
destination_chains

{'positive': LLMChain(prompt=PromptTemplate(input_variables=[], template='positive_template'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002A7950279E0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002A7950507D0>, model_name='gpt-4o-mini', temperature=0.4, openai_api_key=SecretStr('**********'), openai_proxy='')),
 'negative': LLMChain(prompt=PromptTemplate(input_variables=[], template='negative_template'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002A7950279E0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002A7950507D0>, model_name='gpt-4o-mini', temperature=0.4, openai_api_key=SecretStr('**********'), openai_proxy='')),
 'neutral': LLMChain(prompt=PromptTemplate(input_variables=[], template='neutral_template'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002A7950279E0>, async_client=<openai.

In [11]:
destinations =[f"{p['name']}:{p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
print(destinations_str)

positive:Good for analyzing positive sentiments
negative:Good for analyzing negative sentiments
neutral:Godd for analyzing neutral sentiments


In [12]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations= destinations_str)
router_prompt = PromptTemplate(
    template = router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser()
)
router_chain = LLMRouterChain.from_llm(llm=llm, prompt=router_prompt)

chain = MultiPromptChain(router_chain= router_chain,
                         destination_chains=destination_chains,
                         default_chain= destination_chains["neutral"],
                         verbose= True)


In [13]:
chain.run("I ordered pizza for $9.99 and it was awesome!")



> Entering new MultiPromptChain chain...
positive: {'input': 'I ordered pizza for $9.99 and it was awesome!'}
> Finished chain.


'Sure! Here’s a positive template you can use for various purposes, such as emails, messages, or notes of encouragement:\n\n---\n\n**Subject: A Positive Note Just for You!**\n\nDear [Name],\n\nI hope this message finds you in great spirits! I wanted to take a moment to share some positivity and remind you of the incredible things you bring to the world.\n\nYour [specific quality or trait, e.g., kindness, creativity, hard work] truly makes a difference. I admire how you [specific action or achievement, e.g., tackle challenges, support others, inspire those around you]. It’s inspiring to see how you [mention a specific positive outcome or impact].\n\nRemember, every step you take, no matter how small, contributes to your journey and the lives of those around you. Keep shining your light and pursuing your passions!\n\nWishing you a wonderful day filled with joy and success!\n\nWarm regards,\n\n[Your Name]\n\n---\n\nFeel free to customize this template to suit your needs!'

In [14]:
chain.run("What a nasty movie it was. Waste of time!")



> Entering new MultiPromptChain chain...
negative: {'input': 'What a nasty movie it was. Waste of time!'}
> Finished chain.


'It seems like you\'re looking for a "negative template." Could you please clarify what context you need this template for? For example, are you looking for a negative feedback template, a rejection letter, or something else? This will help me assist you better!'